In [118]:
import random
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from dataclasses import dataclass, field

In [119]:
df = pd.read_parquet('./data_matches_synthetic.pq')

In [120]:
def probability_a_wins(r_a, r_b):
    """
    Calculate the probability that player A wins against player B based on their Elo ratings.
    
    The probability is calculated using the formula:
    Pr(A wins) = 1 / (1 + 10^(-(r_A - r_B)/400))
    
    Parameters:
    -----------
    r_a : float
        Elo rating of player A
    r_b : float
        Elo rating of player B
        
    Returns:
    --------
    float
        Probability that player A wins against player B
    """
    r_ab = r_a - r_b
    return 1 / (1 + 10 ** (-r_ab / 400))


In [121]:
def _S(row):
    """Return S_A and S_B based on who is winner"""
    """
    Calculate the score for players A and B based on the match outcome.
    
    Parameters:
    -----------
    row : pandas.Series
        A row from the matches dataframe containing 'player_A', 'player_B', 'winner', and 'loser'
        
    Returns:
    --------
    tuple
        (S_A, S_B) where:
        - S_A = 1 if player A won, 0 if player A lost
        - S_B = 1 if player B won, 0 if player B lost
    """
    if row['winner'] == row['player_A']:
        return 1, 0
    else:
        return 0, 1

In [122]:
# Calculate S_A and S_B for each match
df[['S_A', 'S_B']] = pd.DataFrame(df.apply(_S, axis='columns').tolist(), index=df.index)

In [123]:
all_player_ids = set(list(df['player_A'].unique())) | set(list(df['player_B'].unique()))
player_strength = {k: 0 for k in all_player_ids}

In [124]:
K = 1
for index, row in df.iterrows():
    player_A = row['player_A']
    player_B = row['player_B']
    R_A = player_strength[player_A]
    R_B = player_strength[player_B]
    S_A = row['S_A']
    S_B = row['S_B']
    E_A = probability_a_wins(R_A, R_B)
    E_B = 1 - E_A

    player_strength[player_A] = R_A + K * (S_A - E_A)
    player_strength[player_B] = R_B + K * (S_B - E_B)

In [125]:
player_strength

{'Player_0': -34.047555043779035,
 'Player_2': 40.052199390071216,
 'Player_3': 21.24030004172011,
 'Player_6': -3.2820625952199336,
 'Player_7': -8.244700623490834,
 'Player_8': 13.7028313490912,
 'Player_1': -6.383466847993589,
 'Player_9': -45.792226544390594,
 'Player_4': 9.360583708754335,
 'Player_5': 13.394097165237126}

In [126]:
# Create a DataFrame with player strengths
player_strength_df = pd.DataFrame({
    'player': list(player_strength.keys()),
    'strength': list(player_strength.values())
})

# Sort by strength in descending order
player_strength_df = player_strength_df.sort_values('strength', ascending=False).reset_index(drop=True)

# Display the player strength DataFrame
player_strength_df

,player,strength
0,Player_2,40.052199
1,Player_3,21.240300
2,Player_8,13.702831
3,Player_5,13.394097
4,Player_4,9.360584
5,Player_6,-3.282063
6,Player_1,-6.383467
7,Player_7,-8.244701
8,Player_0,-34.047555
9,Player_9,-45.792227


In [127]:
# Extract player strengths from the dataframe as ground truth
ground_truth = {}

# Add player_A strengths
for player, strength in zip(df['player_A'], df['strength_A']):
    if player not in ground_truth:
        ground_truth[player] = strength

# Add player_B strengths
for player, strength in zip(df['player_B'], df['strength_B']):
    if player not in ground_truth:
        ground_truth[player] = strength

# Create a DataFrame with ground truth player strengths
ground_truth_df = pd.DataFrame({
    'player': list(ground_truth.keys()),
    'strength': list(ground_truth.values())
})

# Sort by strength in descending order
ground_truth_df = ground_truth_df.sort_values('strength', ascending=False).reset_index(drop=True)

# Display the sorted ground truth DataFrame
ground_truth_df.head(10)

,player,strength
0,Player_2,1.867224
1,Player_3,1.243234
2,Player_5,0.979185
3,Player_8,0.898618
4,Player_4,0.707186
5,Player_6,0.637719
6,Player_1,0.465773
7,Player_7,0.449900
8,Player_0,-1.146400
9,Player_9,-1.368363


In [128]:
player_strength_df.head(10)

,player,strength
0,Player_2,40.052199
1,Player_3,21.240300
2,Player_8,13.702831
3,Player_5,13.394097
4,Player_4,9.360584
5,Player_6,-3.282063
6,Player_1,-6.383467
7,Player_7,-8.244701
8,Player_0,-34.047555
9,Player_9,-45.792227
